<center> <img src = './img/hh label.jpg' alt="drawing" style="width:400px;" title='картинка перенесена в локальную папку'>

# <center> Проект: Анализ вакансий из HeadHunter

In [1]:
import pandas as pd
import psycopg2

#удаление предупреждений по вопросам библиотек подключения к Ораклу
import warnings
warnings.filterwarnings("ignore")

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
# поместил параметры во внешнюю по отношению к папке проекта папку
conn_data = pd.read_csv('../../secret_file.txt', sep=':', names=['parameter', 'value'], index_col=[0])

# функция извлечения параметра подключения
def parameter_extract(par):
    par_value = conn_data.loc[par]['value'].strip()
    return par_value

# формирование параметров
DBNAME = parameter_extract(par='DBNAME')
USER = parameter_extract(par='USER')
HOST = parameter_extract(par='HOST')
PASSWORD = parameter_extract(par='PASSWORD')
PORT = parameter_extract(par='PORT')

del conn_data

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
    )

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
    select count(*) as n_valcancies
    from public.vacancies
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
print(df.head())

# Задание 3.1. Сколько вакансий есть в базе?
print(f"Задание 3.1: {df.loc[0]['n_valcancies']}")

   n_valcancies
0         49197
Задание 3.1: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
    select count(*) as n_employers
    from public.employers
'''

In [7]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
print(df.head())

# Задание 3.2. Сколько работодателей?
print(f"Задание 3.2: {df.loc[0]['n_employers']}")

   n_employers
0        23501
Задание 3.2: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
    select count(distinct name) as n_areas
    from public.areas
'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
print(df.head())

# Задание 3.3. Сколько регионов?
print(f"Задание 3.3: {df.loc[0]['n_areas']}, но это же не регионы а города...")

   n_areas
0     1362
Задание 3.3: 1362, но это же не регионы а города...


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
    select count(distinct id) as n_industries
    from public.industries
'''

In [11]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
print(df.head())

# Задание 3.4. Сколько сфер деятельности?
print(f"Задание 3.4: {df.loc[0]['n_industries']}")

   n_industries
0           294
Задание 3.4: 294


***

# выводы по предварительному анализу данных
Согласно базе, в среднем на 1 компанию приходится чуть более 2 (49197 / 23501) вакансий. Не очень много, хотя тут надо бы посмотреть на срерднесписочную численносить персонала  и оценить общий дефицит кадров по выборке.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
    select 
        t1.name,
        count(*) as cnt

    from public.areas as t1
    left join public.vacancies as t2
    on t1.id = t2.area_id
    
    group by t1.name
    order by cnt desc
'''

In [13]:
# результат запроса
# Заданиие 4.1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
# Выберите пятёрку лидеров по количеству вакансий:

# выбор ответа в Задании 4.1 через фильтрацию dataframe:
df = pd.read_sql_query(query_4_1, connection)
print(df.head())

# фильтрация по dataframe - отбор городов с количеством вакансий не меньше, чем в 5 строке, и занесение их в список
city_list = list()

for i in range(df[df['cnt'] >= df['cnt'].iloc[4]].shape[0]): #отбор значений не меньше, чем в пятой строке
    city_list.append(df['name'].iloc[i])

print(f"Задание 4.1: {', '.join(city_list)}")

              name   cnt
0           Москва  5333
1  Санкт-Петербург  2851
2            Минск  2112
3      Новосибирск  2006
4           Алматы  1892
Задание 4.1: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
    select count(*) as cnt
    from public.vacancies
    where coalesce(salary_from, salary_to) is not null
'''

In [15]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
print(df.head())

# Задание 4.2. Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?
print(f"Задание 4.2: {df['cnt'][0]}")

     cnt
0  24073
Задание 4.2: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
    select
        round(avg(salary_from), 0) as avg_min_salary,
        round(avg(salary_to), 0) as avg_max_salary
    from public.vacancies
'''

In [17]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
print(df.head())

# Задание 4.3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.
print(f"""Задание 4.3:
нижняя граница - {int(df['avg_min_salary'][0])},
верхняя граница - {int(df['avg_max_salary'][0])}""")

   avg_min_salary  avg_max_salary
0         71065.0        110537.0
Задание 4.3:
нижняя граница - 71065,
верхняя граница - 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
    select
        schedule,
        employment,
        count(*) as n
        
    from public.vacancies
    group by schedule, employment
    order by count(*) desc
'''

In [19]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
print(df.head())

# Задание 4.4. Напишите запрос, который выведет количество вакансий для каждого сочетания
#типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.
#Какая пара находится на втором месте по популярности?
print(f"Задание 4.4: {df['schedule'].iloc[1]} - {df['employment'].iloc[1]}")

           schedule           employment      n
0       Полный день     Полная занятость  35367
1  Удаленная работа     Полная занятость   7802
2     Гибкий график     Полная занятость   1593
3  Удаленная работа  Частичная занятость   1312
4    Сменный график     Полная занятость    940
Задание 4.4: Удаленная работа - Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
    select
        experience,
        count(*) as n
        
    from public.vacancies
    group by experience
    order by count(*)
'''

In [21]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
print(df.head())

#выбор ответа в Задании 4.5:
experience_list = list()

for i in range(df.shape[0]):
    experience_list.append(df['experience'].iloc[i])

# Задание 4.5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience)
# в порядке возрастания количества вакансий, в которых указан данный вариант опыта.
print(f"Задание 4.5: {' - '.join(experience_list)}")

           experience      n
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152
Задание 4.5: Более 6 лет - Нет опыта - От 3 до 6 лет - От 1 года до 3 лет


***

# выводы по детальному анализу вакансий
Ожидаемо на первых местах по количеству вакансий - крупнешие города, включая столицы государств ближнего зарубежья, в которых не будет языкового барьера
При этом там есть потребность в русскоговорящих спецалистах.
Если взять средние зарплаты (запрос ниже) по регионам с 20+ вакансиями, то в топ-10 российские города вообще не попадут, а в топ-20 таковых будет 8 с Москвой на 11 месте. Уровень зарплаты за границей высокий, и раз ищут на hh, то внутри страны найти не смогли. А если фирмы пытаются сэкономить, найдя российкого специалиста, то сколько же у коренных жителей на аналогичных должностях зарплата...
По условиям работы в базе вне конкуренции - полный день, полная занятость - вакансий такого типа в 5 раз больше, чем у ближайшего преследователя (удаленка). Но интересно замерить динамику: сколько предалагалось удаленки лет 5 назад.
В основном требуются специалисьы с опытом работы 1-3 и 3-6 лет. Это костяк коллектива, замена может быть непростой.
БОльшие стажи - это свехропытные специалисты и руководители, там уже свои каналы поиска - необязательно такое публиковать, ищут через знакомства.

In [22]:
#запрос для выводов
query_test = f'''
    select 
        t1.name,
        count(*) as n,
        avg(coalesce(t2.salary_from, t2.salary_to)) as avg_min,
        avg(coalesce(t2.salary_to, t2.salary_from)) as avg_max
    
    from public.areas as t1
    inner join public.vacancies as t2
    on t1.id = t2.area_id
    
    group by t1.name
    having avg(coalesce(t2.salary_from, t2.salary_to)) > 0
    and count(*) >= 20
    order by avg(coalesce(t2.salary_from, t2.salary_to)) desc
    limit 20
'''
df = pd.read_sql_query(query_test, connection)
print(df.head(20))

                 name     n        avg_min        avg_max
0              Польша    53  257155.666667  257155.666667
1                Кипр   136  209573.205128  237898.230769
2             Армения   223  206207.943396  232931.613208
3              Турция    39  204919.263158  265951.789474
4              Сербия    26  203552.166667  213293.666667
5             Тбилиси   474  184831.987500  229479.325000
6             Кишинёв    23  171897.000000  201121.000000
7           Иннополис    91  170529.411765  181235.294118
8              Батуми    83  168575.777778  231093.111111
9                Баку    91  143905.745098  167588.431373
10             Москва  5333  122274.705402  146005.646357
11            Магадан    25  116506.421053  117559.052632
12         Зеленоград   132  108992.105263  124900.000000
13        Новосибирск  2006   96992.685684  118560.757804
14    Санкт-Петербург  2851   94802.326683  114017.224439
15              Химки    77   91724.117647  105558.823529
16  Аксай (Каз

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
# текст запроса
query_5_1 = f'''
    -- именованный запрос - подсчет количества вакансий по работодателям и сортировка по убыванию количества
    with emp_count as (
        select
            t1.name,
            count(*) as n

        from public.employers as t1
        left join public.vacancies as t2
        on t1.id = t2.employer_id

        group by t1.name
        order by count(*) desc)
    
    -- выбор работодателя / работодателей с наибольшим количеством ваканисий
    select
        name
        from emp_count
        where n in (
            select n from emp_count
            limit 1)
            
    union all
    -- выбор работодателя / работодателей с 4 местом по количеству ваканисий    
    select
        name
        from emp_count
        where n in (
            select n from emp_count
            offset 4 limit 1)
'''

In [24]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
print(df.head())

# Задание 5.1. Напишите запрос, который позволит узнать, какие работодатели находятся
#на первом и пятом месте по количеству вакансий.
# выбор ответа в Задании 5.1:
emp_list = list()

for i in range(df.shape[0]):
    emp_list.append(df['name'].iloc[i])
print(f"Задание 5.1: {' - '.join(emp_list)}")

            name
0         Яндекс
1  Газпром нефть
Задание 5.1: Яндекс - Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:
# текст запроса
query_5_2 = f'''
    -- именованный запрос - регионы и количество работодателей в них (чтоб не пихать как подзапрос в джойнах)
    with employee_count as (
        select
            area,
            count(distinct id) as n_employers
        from public.employers
        group by area
        ),
    
    -- именованный запрос - регионы и количество резюме в них (чтоб не пихать как подзапрос в джойнах)
    vacancy_count as (
        select
            area_id,
            count(*) as n_vacancies
        from public.vacancies
        group by area_id
        )
        
    select
        t1.name,
        coalesce(t2.n_employers, 0) as n_employers,
        coalesce(t3.n_vacancies, 0) as n_vacancies
        
    from public.areas as t1
    left join employee_count as t2
    on t1.id = t2.area
    
    left join vacancy_count as t3
    on t1.id = t3.area_id
'''

#запрос, который выводит только регион для Задания 5.2
query_5_2_sql = f'''
    -- именованный запрос - регионы и количество работодателей в них (чтоб не писать как подзапрос в джойнах)
    with employee_count as (
        select
            area,
            count(distinct id) as n_employers
        from public.employers
        group by area
        ),
    
    -- именованный запрос - регионы и количество резюме в них (чтоб не писать как подзапрос в джойнах)
    vacancy_count as (
        select
            area_id,
            count(*) as n_vacancies
        from public.vacancies
        group by area_id
        )
        
    select
        t1.name,
        coalesce(t2.n_employers, 0) as n_employers,
        coalesce(t3.n_vacancies, 0) as n_vacancies
        
    from public.areas as t1
    left join employee_count as t2
    on t1.id = t2.area
    
    left join vacancy_count as t3
    on t1.id = t3.area_id
    
    where coalesce(t3.n_vacancies, 0) = 0
    order by coalesce(t2.n_employers, 0) desc
    limit 1
'''

In [26]:
# результат запроса
# вывод ответа в Задании 5.2 через dataframe:
df = pd.read_sql_query(query_5_2, connection)
print(df.head())
max_empl_cnt = df[df['n_vacancies'] == 0]['n_employers'].max()
print(f"Задание 5.2 через dataframe: {df[df['n_employers'] == max_empl_cnt]['name'].iloc[0]}")

# вывод ответа в Задании 5.2 через SQL:
df = pd.read_sql_query(query_5_2_sql, connection)
print(f"Задание 5.2 через SQL:  {df['name'].iloc[0]}")

              name  n_employers  n_vacancies
0           Москва         5864         5333
1  Санкт-Петербург         2217         2851
2     Екатеринбург          609         1698
3      Новосибирск          573         2006
4        Австралия            6            0
Задание 5.2 через dataframe: Россия
Задание 5.2 через SQL:  Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
# текст запроса
query_5_3 = f'''
    select
        t1.name,
        count(distinct t2.area_id) as n_regions
    
    from public.employers as t1
    left join public.vacancies as t2
    on t1.id = t2.employer_id
    
    group by t1.name    
    order by count(distinct t2.area_id) desc
'''

#запрос, который выводит только число для Задания 5.3
query_5_3_sql = f'''
    select
        t1.name,
        count(distinct t2.area_id) as n_regions
    
    from public.employers as t1
    left join public.vacancies as t2
    on t1.id = t2.employer_id
    
    group by t1.name    
    order by count(distinct t2.area_id) desc
    limit 1
'''

In [28]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
print(df.head())

# Задание 5.3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.
# Выберите максимальное значение из получившегося списка.
# вывод ответа в Задании 5.3 через dataframe:
df = pd.read_sql_query(query_5_3, connection)

print(f"Задание 5.3 через dataframe: {df['n_regions'].iloc[0]}")

# вывод ответа в Задании 5.3 через SQL:
df = pd.read_sql_query(query_5_3_sql, connection)

print(f"Задание 5.3 через SQL: {df['n_regions'].iloc[0]}")

                     name  n_regions
0                  Яндекс        181
1              Ростелеком        152
2              Спецремонт        116
3  Поляков Денис Иванович         88
4               ООО ЕФИН          71
Задание 5.3 через dataframe: 181
Задание 5.3 через SQL: 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
# текст запроса
query_5_4 = f'''
    select count(*) as n_miss_industry
    from (
        select
            t1.id,
            count(t2.industry_id) as n_industries

        from public.employers as t1
        left join public.employers_industries as t2
        on t1.id = t2.employer_id
        group by t1.id
        ) as t

    where n_industries = 0
'''

In [30]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
print(df.head())

# Задание 5.4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.
# Введите количество, которое вернул запрос, в поле ниже.
print(f"Задание 5.4: {df['n_miss_industry'].iloc[0]}")

   n_miss_industry
0             8419
Задание 5.4: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
# текст запроса
query_5_5 = f'''
    select
        t1.name,
        count(t2.industry_id) as n_industries

    from public.employers as t1
    left join public.employers_industries as t2
    on t1.id = t2.employer_id
    
    group by t1.id
    having count(t2.industry_id) = 4
    
    order by t1.name
    offset 2
    limit 1
'''

In [32]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
print(df.head())

# Задание 5.5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте
# в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.
# Введите в поле ниже название этой компании так же, как оно указано в результате запроса.
print(f"Задание 5.5: {df['name'].iloc[0]}")

   name  n_industries
0  2ГИС             4
Задание 5.5: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
# текст запроса
query_5_6 = f'''
    select
        count(distinct t1.id) as n_software_comp
    
    from public.employers as t1
    left join public.employers_industries as t2
    on t1.id = t2.employer_id
    
    left join public.industries as t3
    on t2.industry_id = t3.id
    
    where t3.name = 'Разработка программного обеспечения'
'''

In [34]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
print(df.head())

# Задание 5.6. С помощью запроса выясните, у какого количества работодателей в качестве
# сферы деятельности указана «Разработка программного обеспечения».
print(f"Задание 5.6: {df['n_software_comp'].iloc[0]}")

   n_software_comp
0             3553
Задание 5.6: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

# извлечение содержимого по тэгу 'table' и классу 'standard sortable'
strings = page.find('table', class_='standard sortable')
# элементы таблицы находятся в теге <td>, причем в таблице 5 столбцов, а первый - номер строки
# тогда надо брать каждый пятый элемент, начиная со второго
city_list = list() #переменная списка городов
for i in range(1, len(strings.find_all('td')), 5):
    city = strings.find_all('td')[i].text
    # у Воронежа есть доп. тег - ссылка в квадратных скобках, дающаая при выводе [а].
    # удаляем это (но вот что еще может добавиться...)
    if city.find('[') >=0:
        city = city[0:city.find('[')]
    
    # добавление в список города
    city_list.append(city)
city_list = tuple(city_list)

In [36]:
# текст запроса
query_5_7 = f'''
-- именованный запрос - кол-во вакансий Яндекса по городам-миллионерам
    with yandex_vacancies as (
        select
            t2.name,
            count(t1.id)::int as n_vacancies

        from public.vacancies as t1
        inner join public.areas as t2
        on t1.area_id = t2.id

        inner join public.employers as t3
        on t1.employer_id = t3.id

        where t2.name in {city_list}
        and upper(t3.name) = 'ЯНДЕКС'

        group by t2.name
        order by count(t1.id)
        )
    select *
    from yandex_vacancies
    
    union all
    
    select
        'Total' as name,
        sum(n_vacancies) as n_vacancies
    from yandex_vacancies    
'''

In [37]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection, index_col='name')
print(df.head(df.shape[0]))

# Задание 5.7. Для компании «Яндекс» выведите список регионов-миллионников,
# в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах.
# Также добавьте строку Total с общим количеством вакансий компании.

# Сколько строк получилось в вашей выборке?
print(f"Задание 5.7: {df['n_vacancies'].shape[0]} строк получилось")

# Какой результат получился в строке Total?
print(f"Задание 5.7: {df['n_vacancies'].loc['Total']} в строке Total")

                 n_vacancies
name                        
Омск                      21
Челябинск                 22
Красноярск                23
Волгоград                 24
Пермь                     25
Казань                    25
Ростов-на-Дону            25
Уфа                       26
Самара                    26
Краснодар                 30
Воронеж                   32
Новосибирск               35
Нижний Новгород           36
Екатеринбург              39
Санкт-Петербург           42
Москва                    54
Total                    485
Задание 5.7: 17 строк получилось
Задание 5.7: 485 в строке Total


***

# выводы по анализу работодателей

Более трети работодателей не указывают отрасль. Это странно, так как так труднее найти их вакансии по желаемым отраслям.
Вообще жля таких случаев можно использовтаь данные пот отрасли по ОКВЭД из СПАРК, например, сопоставив словарю видов экономической деятельности разбивку, используемую на hh.ru
В топе (как по количеству вакансий - запрос ниже), так и по географии - высокотехнологичные компании и финансовый сектор: развивающиеся сектора, большая текучесть персонала.

In [38]:
# текст запроса - топ работодателей по вакансиям всего
query_ = f'''
    select
        t1.name,
        count (t2.name) as n_vacancies
    
    from public.employers as t1
    left join public.vacancies as t2
    on t1.id = t2.employer_id
    
    group by t1.name    
    order by count(t2.name) desc
'''

df = pd.read_sql_query(query_, connection, index_col='name')
print(df.head(10))

                       n_vacancies
name                              
Яндекс                        1933
Ростелеком                     491
Тинькофф                       444
СБЕР                           428
Газпром нефть                  331
ИК СИБИНТЕК                    327
МТС                            292
DataArt                        247
Совкомбанк Технологии          204
Первый Бит                     176


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [39]:
# текст запроса
query_6_1 = f'''
    select
        count(*) as data_vacancies
    from public.vacancies
    where upper(name) like '%DATA%'
    or upper(name) like '%ДАНН%'
'''

In [40]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
print(df.head())

# Задание 6.1. Сколько вакансий имеет отношение к данным?
print(f"Задание 6.1: {df['data_vacancies'].iloc[0]}")

   data_vacancies
0            1771
Задание 6.1: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [41]:
# текст запроса
query_6_2 = f'''
    select
        count(*) as data_vacancies
    from public.vacancies

    where (
        lower(name) like '%data scientist%'
        or lower(name) like '%data science%'
        or lower(name) like '%исследователь данных%'
        or (name like '%ML%' and name not like '%HTML%')
        or lower(name) like '%machine learning%'
        or lower(name) like '%машинн%обучен%'
        )

    and (
        lower(name) like '%junior%'
        or experience = 'Нет опыта'
        or employment = 'Стажировка'
        )
'''

In [42]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
print(df.head())

# Задание 6.2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?
print(f"Задание 6.2: {df['data_vacancies'].iloc[0]}")

   data_vacancies
0              51
Задание 6.2: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [43]:
# текст запроса
query_6_3 = f'''
    select count (id) as n
    from public.vacancies

    where (
        lower(name) like '%data scientist%'
        or lower(name) like '%data science%'
        or lower(name) like '%исследователь данных%'
        or (name like '%ML%' and name not like '%HTML%')
        or lower(name) like '%machine learning%'
        or lower(name) like '%машинн%обучен%'
        )

    and (
        upper(key_skills) like '%SQL%'
        or lower(key_skills) like '%postgres%'        
        )
'''

In [44]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
print(df)

# Задание 6.3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?
print(f"Задание 6.3: {df['n'].iloc[0]}")

     n
0  201
Задание 6.3: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [45]:
# текст запроса
query_6_4 = f'''
    select
        count(*) as data_vacancies
    from public.vacancies

    where (
        lower(name) like '%data scientist%'
        or lower(name) like '%data science%'
        or lower(name) like '%исследователь данных%'
        or (name like '%ML%' and name not like '%HTML%')
        or lower(name) like '%machine learning%'
        or lower(name) like '%машинн%обучен%'
        )

    and lower(key_skills) like '%python%'
'''

In [46]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
print(df.head())

# Задание 6.4. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.
print(f"Задание 6.4: {df['data_vacancies'].iloc[0]}")

   data_vacancies
0             351
Задание 6.4: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [47]:
# текст запроса
query_6_5 = f'''
    --навыки разделены табом (\t). кол-во навыков равно кол-ву табов плюс один
    select
       avg(length(key_skills) - length(replace(key_skills, '\t', '')) + 1) as avg_skills
    from public.vacancies

    where (
        lower(name) like '%data scientist%'
        or lower(name) like '%data science%'
        or lower(name) like '%исследователь данных%'
        or (name like '%ML%' and name not like '%HTML%')
        or lower(name) like '%machine learning%'
        or lower(name) like '%машинн%обучен%'
        )
'''

In [48]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
print(df.head())

# Задание 6.5 Сколько ключевых навыков в среднем указывают в вакансиях для DS?
# Ответ округлите до двух знаков после точки-разделителя.
print(f"Задание 6.5: {round(df['avg_skills'].iloc[0], 2)}")

   avg_skills
0    6.406032
Задание 6.5: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [49]:
# текст запроса
query_6_6 = f'''
    select
        experience,
        --чтобы посчитать среднюю з/п по вакансии, если нет какого-то значения из вилки,
        --то вместо Null на месте отсутствующего значения берем имеющееся.
        --если нет обоих значений, то Null по итогу ((Null+Null)/2) в расчет среднего не возьмется
        round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2), 2) as avg_salsry
    from public.vacancies

    where (
        lower(name) like '%data scientist%'
        or lower(name) like '%data science%'
        or lower(name) like '%исследователь данных%'
        or (name like '%ML%' and name not like '%HTML%')
        or lower(name) like '%machine learning%'
        or lower(name) like '%машинн%обучен%'
        )
    group by experience
    order by experience
'''

In [50]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
print(df.head())

print(f"Задание 6.6: {int(round(df[df['experience'] == 'От 3 до 6 лет']['avg_salsry'].iloc[0], 0))}")

           experience  avg_salsry
0         Более 6 лет         NaN
1           Нет опыта    74642.86
2  От 1 года до 3 лет   139674.75
3       От 3 до 6 лет   243114.67
Задание 6.6: 243115


***

# выводы по предметному анализу

Профессия дата-сайентиста очень неплохо оплачивается. Даже без опыта средняя зарплата составляет 75 000 рублей (по сути очень хорошо оплачиваемая стажировка - набор опыта, да еще и дают неплохие деньги), за год-полтора работы ставка удваивается, а еще через полтора-два года утраивается от начальной. У спецов от 6 лет зарплату даже и не пишут - чтоб не расстраивать топ-менеджеров Газпрома и Сбербанка, не иначе.

Язык python является must have-навыком: он требуется почти в 75% вакансий (351/480, где 480 - вакансий на дата-сайентиста всего из запроса query_6_add_1 ниже). Так что учить его полезно и выгодно в перспективе (но это пока [ChatGPT](https://www.youtube.com/watch?v=bPKw6fY23SU) не успел вытеснить...)

In [51]:
# текст запроса - количество вакансий дата-сайентиста всего
query_6_add_1 = f'''
    select
        count(*) as data_vacancies
    from public.vacancies

    where (
        lower(name) like '%data scientist%'
        or lower(name) like '%data science%'
        or lower(name) like '%исследователь данных%'
        or (name like '%ML%' and name not like '%HTML%')
        or lower(name) like '%machine learning%'
        or lower(name) like '%машинн%обучен%'
        )
'''
df = pd.read_sql_query(query_6_add_1, connection)
print(df.head())

   data_vacancies
0             480


# Общий вывод по проекту

In [52]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, 
# сделать прогнозы, продумать варианты продолжения исследования

Интересный срез рынка труда. Можно сравнить оплату по вакансиям с оплатой в своей текущей сфере работы.

Востребованность программистов python растет, зарплаты в сфере высокие: чтобы получать хорошие деньги, нужно выучить язык, попасть на начальную похицию и уже через год-два с портфолио выполненных проектов получать достойную зарплату.
Интересно посмотреть динамику зарплаты во времени, а также доли вакансий с python в требованиях.

Всьма логично получилось распределение по количеству вакансий и по средней предлагаемой зарплате.
Чтобы много зарабатывать, нужно искать работу в столицах. Но там же мест и больше всего.

In [53]:
connection.close()